import modules

In [7]:
import numpy as np
import onsager.crystal as crystal

A Pair object (that represents a dumbbell-solute state) should have the following attributes:
1. It should have the locations of the solute and dumbbell.
2. A pair object contains information regarding which atom in it is going to jump.
3. We should be able to apply Group operations to it to generate new pairs.
4. We should be able to add a Jump object to a Pair object to create another pair object.
5. We should be able to test for equality.

In [10]:
class pairs(object):
    
    def __init__(self,crys,chem,i_s,i_d,R_s,R_d,c,ptr):
        '''
            param: crys - crystal object on which diffusion will occur.
            param: chem - Index of sublattice (in crys) on which diffusion will occur.
            param: i_s,i_d : basis indices of solute and dumbbell
            param: R_s,R_d : lattice vectors for solute and dumbbell
            param: c - orientation vector of the dumbbell (centered at dumbbell site).
            param: ptr - (+-1) - points to the "jumping" atom in the dumbbell. +1 means the atom at the head
                   of the vector "c", -1 means the atom at the tail of the vector "c".
                   In case of mixed dumbbell, ptr=1 means that solute will be jumping, which means mixed dumbbell jump.
        '''
        self.crys = crys
        self.chem = chem
        self.i_s = i_s
        self.i_d = i_d
        self.R_s = R_s.copy()
        self.R_d = R_d.copy()
        self.c = c.copy()
        self.ptr = ptr
    
    def __eq__(self, other):
        true_class = isinstance(other,self.__class__)
        true_vectors = np.allclose(self.R_s,other.R_s) and np.allclose(self.c,other.c)
        true_indices = (self.i_s == other.i_s and self.i_s == other.i_s and self.ptr == other.ptr)
        return (true_class and true_vectors and true_indices)
    
    def __add__(self,other):
        if not isinstance(other,Jumps):
            raise TypeError("Can only add a jump to a state. Make sure they have same crystal and chemistry")
            #should I put an additional check to see if the crystal and chemistry are the same?
        if not (self.i_s==other.i and self.ptr==other.ptr1 and np.allclose(self.R_d,other.R1) and np.allclose(self.c,other.c1)):
            raise ArithmeticError("Initial state of Jumps object must match current dumbbell state")
        if self.i_s==self.i_d and np.allclose(self.R_s,self.R_d) and self.ptr==1: #meaning mixed dumbbell jump
            return self.__class__(self.crys,self.chem,other.j,other.j,other.R2,other.R2,other.c2,1)
        else: #solute remains fixed, dumbbell jumps
            return self.__class__(self.crys,self.chem,self.i_s,other.j,self.R_s,other.R2,other.c2,self.ptr)
    
    def g_op(self,g):#apply group operation
        R_s_new, (c,i_s_new) = self.crys.g_pos(g,self.R_s,(self.chem,self.i_s))
        R_d_new, (c,i_d_new) = self.crys.g_pos(g,self.R_d,(self.chem,self.i_d))
        c_new = np.dot(g.cartrot,self.c)
        return self.__class__(self.crys,self.chem,i_s_new,i_d_new,R_s_new,R_d_new,c_new,self.ptr)
        

Jump objects are rather simple, contain just initial and final orientations, location and pointers towards tagged atom (+1 for head of orientation vector, -1 for tail of orientation vector).

In [11]:
class Jumps(object):
    
    def __init__(self,crys,i,j,R1,R2,dx,c1,c2,ptr1,pt2):
        '''
            param: i - initial basis index of dumbbell
            param: j - final basis index of dumbbell
            param: R1 - intial lattice vector of the dumbbell
            param: R2 - final lattice vector of the dumbbell
            param: c1,c2 - initial,final orientations of the dumbbell
            param: ptr1,ptr2 - initial,final final pointers towards tagged atom.
        '''
        self.crys = crys
        self.i = i
        self.j = j
        self.R1 = R1.copy()
        self.R2 = R2.copy()
        self.dx = self.crys.pos2cart(R2, j) - self.crys.pos2cart(R1, i)
        self.c1 = c1.copy()
        self.c2 = c2.copy()
        self.ptr1 = ptr1
        self.ptr2 = ptr2

General workflow:
1. Generate a jumpnetwork containing transitions.
2. Convert the transitions to Jumps objects.
3. Add the Jumps objects to existing Pairs to generate new Pairs.